<a href="https://colab.research.google.com/github/bessabra/PRO2PRO/blob/main/ListaSeiRevisao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [114]:
# !pip install fuzzywuzzy
# !pip install levenshtein

In [115]:
import pandas as pd
from fuzzywuzzy import fuzz
from datetime import datetime, timedelta

basedf = pd.read_excel('/content/Prof Pioneiros Referencia tudo.xlsx', sheet_name='285715-CNV')


In [ ]:
#Lista SEI de requerimentos para selecionar 40801 40803 40804

baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_COMPLEMENTO_40804_09_01_2023.xlsx')
# baserq = pd.read_excel('/content/LISTA_SEI_PROFESSORES_COMPLEMENTO_40801_09_01_2023.xlsx')

In [131]:
#Altera o nome da coluna "Siape" para "SIAPE", dessa forma o nome ficará igual entre a basedf e baserq
basedf = basedf.rename(
    columns = {'Siape':'SIAPE'}
    )
baserq['INTERESSADO'] = baserq['INTERESSADO'].str.upper()

In [132]:
#Habilitar linha de acordo com o alvo

#Retira todo servidor de orgão diferente de 40801
# basedf = basedf[basedf['Órgão'] == 40801]

#Retira todo servidor de orgão diferente de 40803
# basedf = basedf[basedf['Órgão'] == 40803]

#Retira todo servidor de orgão diferente de 40804
basedf = basedf[basedf['Órgão'] == 40804]

In [133]:
#Corrige os números SIAPE dos servidores MARIA JOSÉ BARBOSA DA SILVA, ANA HELENA FIGUEIREDO SILVA, FRANCISCO SILVA CUSTÓDIO e CIPRIANA DOS SANTOS MACEDO em 40801.

# Dicionário com as informações de mudança
substituicoes = {10109595:1009595, 101117:1011117, 101527:1011527, 1111456:1011456}

#Substituindo
for valor_antigo, valor_novo in substituicoes.items():
    baserq.loc[baserq['SIAPE'] == valor_antigo, 'SIAPE'] = valor_novo

In [134]:
#Corrige os números SIAPE dos servidores em 40804. 
# JOSUÉ DA SILVA, OSBELTO RIBEIRO TRINDADE, SILVANA RODRIGUES DA SILVA, 
# MIRIAN FERNANDES DE MELO, ANA AMELIA ALMEIDA SILVA, MARILENE EUSEBIO TOME MUNHOZ, 
# ELANI DO SOCORRO FERNANDES PINHEIRO STUART, MARLEDE C.SCHWAB e ROSANGELA DA COSTA WERMOUTH 

# Dicionário com as informações de mudança
substituicoes = {700955:709955, 
                 1149046:707477, 
                 6713577:713577, 
                 7058489:705848,
                 7095619:709561, 
                 7115326:711532, 
                 7135882:713588, 
                 10365133:1036513, 
                 1148996:707406}

#Substituindo
for valor_antigo, valor_novo in substituicoes.items():
    baserq.loc[baserq['SIAPE'] == valor_antigo, 'SIAPE'] = valor_novo


In [135]:
# Junta as planilhas basedf e baserq pra identificar correspondência de SIAPE
mixResultadoGeral = baserq.merge(
    basedf, on=['SIAPE'], 
    how='outer', 
    # how='right', 
    suffixes=['', '_'], 
    indicator=True
    )


# Adiciona uma coluna com legendas para cada situação
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only'), 'CONDIÇÃO']   = 'Número SIPAE não encontrado em Prof Pioneiros Referencia tudo'
mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both'), 'CONDIÇÃO']        = 'Número SIPAE encontrado em Prof Pioneiros Referencia tudo'
# Lista de colunas para manter
colunas_manter = ['PROCESSO SEI', 
                  'DATA DO REQUERIMENTO', 
                  'SIAPE', 
                  'INTERESSADO',
                  'ASSUNTO',
                  'GR-MATRICULA',
                  'Órgão',
                  'IT-NU-IDEN-SERV-ORIGEM',
                  'IT-NO-SERVIDOR',
                  'IT-NU-CPF',
                  'CONDIÇÃO', 
                  '_merge',
                  'IT-CO-CARGO-EMPREGO']

# Exclusão das colunas que não estão na lista
mixResultadoGeral = mixResultadoGeral[colunas_manter]

# Cria uma nova planilha somente com os servidores não encontrados na Prof Pioneiros Referencia tudo
servNao = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'left_only')]

# Cria uma nova planilha somente com os servidores encontrados Prof Pioneiros Referencia tudo
servEnc = mixResultadoGeral.loc[(mixResultadoGeral['_merge'] == 'both')]


In [ ]:
#Calcula a similaridade entre os nomes dos interessados e dos servidores. Indentificar se o servidor foi o requerente.
servEnc["SIMILARIDADE"] = servEnc.apply(lambda row: fuzz.partial_ratio(str(row["INTERESSADO"]), str(row["IT-NO-SERVIDOR"])) if (isinstance(row["INTERESSADO"],str) and isinstance(row["IT-NO-SERVIDOR"],str)) else None, axis=1)


#Exclue colunas de merge
servNao.drop('_merge', axis = 1, inplace=True)
servEnc.drop('_merge', axis = 1, inplace=True)

In [138]:
#Gerando as planilhas
servNao.to_excel('RevisaoListaSeiV22.01.07.xlsx')
servEnc.to_excel('EncontradosRefTudoSimilaridadev22.01.07.xlsx')